In [ ]:
%pip install unsloth

In [ ]:
%pip install tf-keras

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch

model_name = "Qwen/Qwen3-4B-Instruct-2507"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None,
    device_map="auto",
)


In [ ]:
from datasets import load_dataset

ds = load_dataset("xlangai/spider")

def format_sample(sample):
    return {
        "text":
        f"""### Instruction:
Convert the question into an SQL query for the database: {sample['db_id']}.

### Question:
{sample['question']}

### SQL:
{sample['query']}"""
}

ds = ds.map(format_sample)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "v_proj", "k_proj", "o_proj"],
)


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="qwen-spider-output",
    num_train_epochs=1,                
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    warmup_steps=20,
    logging_steps=20,

    save_strategy="epoch",              
    save_total_limit=3,

    fp16=True,
    bf16=False,
    report_to="none",
)


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    dataset_text_field="text",
    max_seq_length=2048,
    args=args,
)


In [ ]:
trainer.train()

In [ ]:
trainer.train(
    resume_from_checkpoint="qwen-spider-output/checkpoint-438"
)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="qwen-spider-output",
    num_train_epochs=2,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    warmup_steps=20,

    save_strategy="epoch",
    save_total_limit=3,

    logging_steps=20,

    fp16=True,
    bf16=False,
    report_to="none",

    remove_unused_columns=False,
)


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    dataset_text_field="text",
    max_seq_length=2048,
    args=args,
)


In [ ]:
trainer.train(
    resume_from_checkpoint="qwen-spider-output/checkpoint-438"
)


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="qwen-spider-output",
    num_train_epochs=3,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    warmup_steps=20,

    save_strategy="epoch",
    save_total_limit=3,

    logging_steps=20,

    fp16=True,
    bf16=False,
    report_to="none",

    remove_unused_columns=False,
)


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    dataset_text_field="text",
    max_seq_length=2048,
    args=args,
)


In [ ]:
trainer.train(
    resume_from_checkpoint="qwen-spider-output/checkpoint-876"
)


In [ ]:
# Save LoRA adapter
model.save_pretrained("qwen-spider-lora")
tokenizer.save_pretrained("qwen-spider-lora")

# Merge LoRA into base model
merged_model = model.merge_and_unload()

# Save merged model
merged_model.save_pretrained("qwen-spider-merged", safe_serialization=True)
tokenizer.save_pretrained("qwen-spider-merged")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model_name = "Qwen/Qwen3-4B-Instruct-2507"
lora_path = "qwen-spider-lora"   # your saved LoRA adapter

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

model = PeftModel.from_pretrained(
    base_model,
    lora_path,
)

model.eval()


In [ ]:
prompt = """
### Instruction:
Convert the question into an SQL query using the given database schema.

### Database:
dog_kennels

### Schema:
dogs(
  dog_id INTEGER,
  dog_name TEXT,
  kennel_id INTEGER
)

kennels(
  kennel_id INTEGER,
  kennel_name TEXT
)

### Question:
List dogs that do not have a kennel_id (i.e., kennel_id is NULL).

### SQL:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.0,
        do_sample=False,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
